In [11]:
import sys
import pandas as pd

# Alternate between ../../ and ../../src due to bug
sys.path.insert(0,'../../src')

from src.service.exchange import ExchangeService
from src.config import app_config, ProductionConfig
from src.enums import ExchangeType
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

ENGINE  = create_engine(app_config.SQLALCHEMY_DATABASE_URI, echo=False, pool_size=20, max_overflow=0)

prod_config = ProductionConfig()

demo = ExchangeService().get_exchange(ExchangeType.BINANCE.value, app_config.BINANCE_API_KEY, app_config.BINANCE_API_SECRET)
prod = ExchangeService().get_exchange(ExchangeType.BINANCE.value, prod_config.BINANCE_API_KEY, prod_config.BINANCE_API_SECRET)

# Checking if the values are the same
print(demo.get_symbol_ticker('ETHUSDT'))
print(prod.get_symbol_ticker('ETHUSDT'))

{'symbol': 'ETHUSDT', 'price': '1829.79000000'}
{'symbol': 'ETHUSDT', 'price': '1829.79000000'}


In [22]:
# markets = prod.load_markets()
df = pd.DataFrame(prod.get_all_symbols()).T
df.head()

,symbol,timestamp,datetime,high,low,bid,bidVolume,ask,askVolume,vwap,open,close,last,previousClose,change,percentage,average,baseVolume,quoteVolume,info
ETH/BTC,ETH/BTC,1684135880666,2023-05-15T07:31:20.666Z,0.067392,0.0665,0.066723,24.6201,0.066724,13.3308,0.066983,0.067278,0.066723,0.066723,0.067279,-0.000555,-0.825,0.067001,49168.7781,3293.477347,"{'symbol': 'ETHBTC', 'priceChange': '-0.000555..."
LTC/BTC,LTC/BTC,1684135879717,2023-05-15T07:31:19.717Z,0.003202,0.003007,0.003177,32.683,0.003178,125.272,0.003102,0.003024,0.003177,0.003177,0.003024,0.000153,5.06,0.0031,143457.142,444.932087,"{'symbol': 'LTCBTC', 'priceChange': '0.0001530..."
BNB/BTC,BNB/BTC,1684135876655,2023-05-15T07:31:16.655Z,0.011662,0.011461,0.011496,17.471,0.011497,0.576,0.011569,0.011653,0.011496,0.011496,0.011653,-0.000157,-1.347,0.011574,28379.202,328.318209,"{'symbol': 'BNBBTC', 'priceChange': '-0.000157..."
NEO/BTC,NEO/BTC,1684135877542,2023-05-15T07:31:17.542Z,0.000339,0.000333,0.000337,559.4,0.000338,652.3,0.000336,0.000337,0.000337,0.000337,0.000337,0.0,0.0,0.000337,35819.55,12.049442,"{'symbol': 'NEOBTC', 'priceChange': '0.0000000..."
QTUM/ETH,QTUM/ETH,1684135804959,2023-05-15T07:30:04.959Z,0.001424,0.001409,0.001415,77.4,0.001419,207.4,0.001418,0.001409,0.001412,0.001412,0.001418,0.000003,0.213,0.001411,235.2,0.333588,"{'symbol': 'QTUMETH', 'priceChange': '0.000003..."


### Generating load test data

##### Users
Creates N users.

In [21]:
import random
import string

from src.model import UserModel

def randomString(stringLength=10) -> str:
    letters = string.ascii_letters
    return ''.join([random.choice(letters) for i in range(stringLength)])

number_of_users = 1000

with Session(ENGINE) as session:
    for i in range(number_of_users):
        rnd = randomString()
        user = UserModel(
            email=rnd + '@gmail.com',
            password=rnd,
        )
        session.add(user)
        session.flush()

    session.commit()

##### Symbols & Currency pairs

In [26]:
target_currency = "USDT"

existing_symbols = {'BTC/USDT', 'ETH/USDT'}
invalid_symbols = {'YOYOW/ETH', 'YOYOW/BNB', 'YOYOW/BTC', 'BTCDOM/USDT', 'LUNA2/USDT', 'LUNA/USDT', 'BLUEBIRD/USDT', 'DEFI/USDT', '1000SHIB/USDT', 'MANA/USDT', '1000LUNC/USDT', 'BLUR/USDT', 'FOOTBALL/USDT', '1000PEPE/USDT', '1000FLOKI/USDT', '1000XEC/USDT', 'NBTUSDT'}

avoid = existing_symbols.union(invalid_symbols)

# Get all the symbols
currency_pairs = set(df["symbol"].to_list())
currency_pairs = [symbol for symbol in currency_pairs if symbol.endswith(target_currency) and symbol not in avoid]
currency_pairs[:5]

['BEAM/USDT', 'STG/USDT', 'KSM/USDT', 'UNIDOWN/USDT', 'LPT/USDT']

In [27]:
symbol_query = "INSERT INTO symbols (symbol) VALUES ('{}');"
symbol_queries = [symbol_query.format(currency_pair.split('/')[0]) for currency_pair in currency_pairs]

currency_pair_queries = []
for currency_pair in currency_pairs:
    currency_pair_clean = currency_pair.replace('/', '')

    try:
        symbol = currency_pair.split('/')[0]
        pair = currency_pair.split('/')[1]
        currency_pair_queries.append(f"INSERT INTO currency_pairs (symbol, pair, currency_pair) VALUES ('{symbol}', '{pair}', '{currency_pair_clean}');")
    except:
        print(currency_pair)

with open('symbols.load.sql', 'w+') as f:
    for query in symbol_queries:
        f.write(query + '\n')
    
    for query in currency_pair_queries:
        f.write(query + '\n')

## Bot generation

In [4]:
import sys
sys.path.append('../../')

from src.config import ProductionConfig

app_config = ProductionConfig()

from src.enums import StrategyType, ExchangeType, Interval
from src.model import UserModel, ApiKeyModel, PortfolioModel, BalanceModel, CurrencyPairConfigModel, SignalModel, StrategyConfigModel

In [5]:
# Queries
user_query = "INSERT INTO users (id, email, _password) VALUES ('{}', '{}', '{}');"

api_key_query = "INSERT INTO api_keys (user_id, exchange, api_key, api_secret) VALUES ('{}', '{}', '{}', '{}');"

portfolio_query = "INSERT INTO portfolios (id, user_id) VALUES ({}, '{}');"

balance_query = "INSERT INTO balances (portfolio_id, asset, free, locked, total) VALUES ('{}', '{}', {}, {}, {});"

currency_pair_config_query = "INSERT INTO currency_pair_configs (user_id, currency_pair, bot_name, exchange, `interval`, strategy, `limit`, is_active, stop_loss, take_profit, allocated_balance, currency_free, currency_locked, asset_free, asset_locked) VALUES ('{}', '{}', '{}', '{}', '{}', '{}', 1000, 1, 0.3, 0.3, {}, {}, 0.0, 0.0, 0.0);"

signal_query = "INSERT INTO signals (currency_pair_config_id, `signal`) VALUES ({}, 'HOLD');"

strategy_config_query = "INSERT INTO strategy_configs (currency_pair_config_id, strategy, `key`, `value`) VALUES ({}, '{}', '{}', {});"

In [6]:
bot_config = {
    'email': 'bot{}@admin.com',
    'password': '$2b$12$F/EmqrDOVllQXDkO9xBXbexbwXVit/GuXOxSvAiQ3NrvPSwJCv7VK',
    'name': 'Bot {} - {}',
    'starting_currency': 5000.0,
    'symbols': ['MINA', 'AXS', 'TRU', 'CKB', 'EDU'],
    'currency': 'USDT',
    'api_key': app_config.BINANCE_API_KEY,
    'api_secret': app_config.BINANCE_API_SECRET,
    'exchange': ExchangeType.BINANCE.name
}

strategies = [
    {
        'strategy': StrategyType.RSI.name,
        'params': {'window': 1},
        'interval': Interval.MINUTE_1.name
    },
]

In [6]:
# Create user
user_queries = []
# Create api key
api_key_queries = []
# Create portfolio
portfolio_queries = []
# Create balances
balance_queries = []
# Create currency pair config
currency_pair_config_queries = []
# Create signal
signal_queries = []
# Create strategy config
strategy_config_queries = []

starting_user_id = 47
config_id = 942 # starting config id
for i, strategy in enumerate(strategies):
    id = starting_user_id + i
    strategy_type = strategy['strategy']
    params = strategy['params']
    interval = strategy['interval']

    # Create user
    query = user_query.format(id, bot_config['email'].format(id), bot_config['password'])
    user_queries.append(query)

    # Create api key
    query = api_key_query.format(id, bot_config['exchange'], bot_config['api_key'], bot_config['api_secret'])
    api_key_queries.append(query)

    # Create portfolio
    query = portfolio_query.format(id, id)
    portfolio_queries.append(query)

    # Create balances
    query = balance_query.format(
        id,
        'USDT',
        0.0,
        bot_config['starting_currency'] * len(bot_config['symbols']),
        bot_config['starting_currency'] * len(bot_config['symbols'])
    )
    balance_queries.append(query)
    for symbol in bot_config['symbols']:
        query = balance_query.format(id, symbol, 0.0, 0.0, 0.0)
        balance_queries.append(query)

        # Create currency pair config
        query = currency_pair_config_query.format(
            id,
            f"{symbol}{bot_config['currency']}",
            bot_config['name'].format(id, symbol),
            bot_config['exchange'],
            interval,
            strategy_type,
            bot_config['starting_currency'],
            bot_config['starting_currency']
        )
        currency_pair_config_queries.append(query)
    
        # Create signal
        query = signal_query.format(config_id)
        signal_queries.append(query)

        # Create strategy config
        for key, value in params.items():
            query = strategy_config_query.format(config_id, strategy_type, key, value)
            strategy_config_queries.append(query)
        
        config_id += 1
        

with open('bot.testing.sql', 'w+') as f:
    for query in user_queries:
        f.write(query + '\n')
    
    for query in api_key_queries:
        f.write(query + '\n')
    
    for query in portfolio_queries:
        f.write(query + '\n')
    
    for query in balance_queries:
        f.write(query + '\n')
    
    for query in currency_pair_config_queries:
        f.write(query + '\n')
    
    for query in signal_queries:
        f.write(query + '\n')
    
    for query in strategy_config_queries:
        f.write(query + '\n')

## SQL Debugging and fixing

In [28]:
from src.service import DataService

##### Updating invalid strategy in the strategy configs

In [29]:
with Session(ENGINE) as session:
    bots = DataService().get_bots(session, '45')
    for bot in bots:
        configs = bot.strategy_config
        for config in configs:
            config.strategy = StrategyType.CHAIKIN_OSCILLATOR.name
            # session.flush()

    # session.commit()

##### Deleting invalid symbols

In [21]:
with ENGINE.connect() as conn:
    for currency_pair in invalid_symbols:
        symbol = currency_pair[:-4]
        conn.execute(text(f"DELETE FROM symbols WHERE symbol = '{symbol}'"))
        conn.execute(text(f"DELETE FROM currency_pair_configs WHERE currency_pair = '{currency_pair}'"))
        conn.commit()